![logo](img/logo.png)
# Best Practices for the u2py.File object

#### Note if you are having problems with any of these examples, please see the [Troble shooting u2py](Trouble_Shooting_u2py.ipynb#Main)


In [1]:
import os
xdemoPath = "c:\\U2\\ud83\\XDEMO"
os.chdir(xdemoPath)
print(os.getcwd())
import u2py

c:\U2\ud83\XDEMO


<a id='u2py_file_read'></a>
## The u2py.File.Read menthod best practice

For the Rocket MultiValue BASIC developer who is use to the read statement that either reads the item or sets the variable to an empty dynamic array.

i.e. Of the command in BASIC
READ REC FROM FILEVAR, ID ELSE REC = ""

The following function is part of the u2support.py module, wich is documented in detail in the [u2support.py module doc](u2py_File_Best_Practices.ipynb#U2READ).

In [10]:
def U2READ(theFile, itemId, defaultItem):
    try:    
        if type(theFile).__name__ == 'File':
            myFile = theFile
        else:
            myFile = u2py.File(theFile)


        rec = myFile.read(itemId)

    except u2py.U2Error:
        if type(defaultItem) == u2py.DynArray:
            rec = defaultItem
        else:
            rec = u2py._DynArray(defaultItem)

    return rec

myrec = U2READ("STATES", "CA", "")



### Testing the U2READ

Modify the parameters used in the funcion, and see how it works.

In [6]:
test = U2READ("MEMBERS", "not an item", "no item read")
print(test)

no item read


## Ways to display the dynamic array

GJDHGKDHGKGKJ

In [11]:
print("Print the dynamic array as a string")
print(str(myrec))
print("")

print("print each element of the dynamic array")
for each_value, delim in myrec:
    print(str(each_value))
print("")
    
print("Casting it to a Python list")
mylist = myrec.to_list()
print(mylist)
print("")

print("Join the list with newline characters")
joined = "\n".join(mylist)
print(joined)

Print the dynamic array as a string
California�West�Pacific

print each element of the dynamic array
California
West
Pacific

Casting it to a Python list
['California', 'West', 'Pacific']

Join the list with newline characters
California
West
Pacific


## A better way of reading and writing data

A better way to read and write data to U2 from Python is with the readnamedfields and writenamedfields 
methods of the u2py.File object.

In [14]:
help(u2py.File.readnamedfields)

Help on function readnamedfields in module u2py:

readnamedfields(self, *args)
    F.readnamedfields(recordid, fieldnnames, [lockflag]) -> new DynArray object -- read the specified fields by name of a record in the file
    
    fieldnames is a u2py.DynArray object with each of its fields being a name defined in the dictionary file
    lockflag is either 0 (default), or [LOCK_EXCLUSIVE or LOCK_SHARED] [ + LOCK_WAIT]
    
    note: if fieldnames contains names that are not defined in the dictionary, these names are replaced by @ID and no exception is raised



## Building a helper method arund the u2py.File.readnamedfields method

In [12]:
def u2ReadPyDict( theFile, itemId, fieldList ):
    if type(theFile).__name__ == 'File':
        myFile = theFile
    else:
        myFile = u2py.File(theFile)

    if type(fieldList).__name__ == 'list':
        fieldDynArray = u2py.DynArray(fieldList)
    else:
        raise ValueError("Expecting Python list of field names")
        return null

    notFound = 0
    try:
        rec = myFile.readnamedfields(itemId, fieldDynArray)
        rec_list = rec.to_list()
        my_dict = dict(zip(fieldList, rec_list))
    except u2py.U2Error as e:
        print( "item " + str(itemId) + " was not read from the file")
        print( str(e))

    return my_dict

With the above method, we are able to get a Python dictionary item that contains the fields we requested.

In [ ]:
from pprint import pprint
fieldlist = ["FIRST_NAME", "LAST_NAME", "BIRTHDATE"]
stateInfo = u2ReadPyDict("MEMBERS", "0110", fieldlist)
pprint(stateInfo)
print(stateInfo)

{'BIRTHDATE': '01/25/1982', 'FIRST_NAME': 'Anita', 'LAST_NAME': 'Johnson'}
{'FIRST_NAME': 'Anita', 'LAST_NAME': 'Johnson', 'BIRTHDATE': '01/25/1982'}


##### Note that the birth date is the OCONV (output), and not the inernal value that is stored in the database.

In [27]:
rec = U2READ("MEMBERS", "0110", "")
# convert dynamic array to list
recList = rec.to_list()
# since python list starts at location 0, attribut 10 is found at location 9
print(recList[9])

5139


# The u2support.py module has the u2WritePyDict method, to write the dictionary back to U2.

#### Note that it will write back the internal (ICONV) value for any D-typ dictionary that has a conversion code.

In [28]:
def u2WritePyDict( theFile, itemId, theDictItem ):
    if type(theFile).__name__ == 'File':
        myFile = theFile
    else:
        myFile = u2py.File(theFile)

    if type(theDictItem).__name__ == 'dict':
        fieldNames, fieldValues = zip(*theDictItem.items())
        fieldNames = u2py.DynArray(list(fieldNames))
        fieldValues = u2py.DynArray(list(fieldValues))
    else:
        raise ValueError("Expecting Python dict item for theDictItem")
        return null

    myFile.writenamedfields(itemId,fieldNames, fieldValues )
    return

The following example, shows how we can change a value, and write it back to U2.

In [37]:
fieldlist = ["FIRST_NAME", "LAST_NAME", "BIRTHDATE"]
member = u2ReadPyDict("MEMBERS", "0110", fieldlist)
print(member["FIRST_NAME"] + " " +member["LAST_NAME"] + "'s birthdate is " + member["BIRTHDATE"])
newbirthdate = input("Enter a new birthdate")
member["BIRTHDATE"] = newbirthdate
u2WritePyDict("MEMBERS", "0110", member)

Anita Johnson's birthdate is 12/25/1980
